In [ ]:
# Cài đặt dependencies
import subprocess
import sys

def install_requirements():
    """Cài đặt các package cần thiết."""
    packages = [
        "unsloth",
        "--upgrade timm",  # Cho Gemma 3N
        "comet-ml"
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install"] + package.split())
            print(f"✅ Installed {package}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}: {e}")

install_requirements()


In [ ]:
# Imports
import os
import re
import io
from typing import Tuple, List, Dict, Any, Optional
from PIL import Image
import requests
import torch
from datasets import load_dataset, Dataset
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig
from unsloth import FastVisionModel, get_chat_template

# Comet ML (optional)
try:
    import comet_ml
    COMET_AVAILABLE = True
    print("🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.")
except ImportError:
    COMET_AVAILABLE = False
    print("⚠️ comet_ml không có sẵn. Logging sẽ chỉ dùng tensorboard.")


In [ ]:
# Configuration tối ưu cho mobile deployment
CONFIG = {
    # Model settings - E2B tốt hơn cho mobile (nhỏ hơn, ít tham số hơn)
    "model_name": "unsloth/gemma-3n-E2B",  # Thay đổi từ E4B sang E2B
    "max_seq_length": 1024,  # Giảm từ 2048 để tối ưu memory cho mobile
    "load_in_4bit": True,

    # Dataset settings
    "dataset_name": "ngohongthai/exam-sixth_grade-instruct-dataset",
    "train_split": "train",

    # Training settings - tối ưu cho mobile
    "output_dir": "./gemma3n_e2b_sixth_grade",
    "max_steps": 100,  # Giảm số steps để training nhanh hơn
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 4,  # Giảm từ 8
    "learning_rate": 5e-5,  # Tăng learning rate một chút
    "warmup_ratio": 0.1,  # Tăng warmup
    "weight_decay": 0.01,
    "logging_steps": 10,
    "save_steps": 25,

    # LoRA settings - tối ưu cho mobile
    "lora_r": 16,  # Giảm từ 32
    "lora_alpha": 16,  # Giảm từ 32
    "lora_dropout": 0.05,  # Thêm một chút dropout

    # System settings
    "use_gradient_checkpointing": True,  # Enable để tiết kiệm memory
    "report_to": "comet_ml" if COMET_AVAILABLE else "tensorboard",
    "seed": 42,
    
    # Mobile optimization flags
    "optimize_for_mobile": True,
    "target_device": "mobile"
}

# Comet ML settings (optional)
COMET_CONFIG = {
    "api_key": os.getenv("COMET_API_KEY"),
    "workspace": os.getenv("COMET_WORKSPACE", "default"),
    "project": os.getenv("COMET_PROJECT", "gemma3n-sixth-grade"),
    "experiment_name": "gemma3n_e2b_mobile_optimized",
    "tags": [
        "gemma3n-e2b",
        "multimodal",
        "math-tutor",
        "vietnamese",
        "sixth-grade",
        "mobile-optimized"
    ]
}

print(f"📱 Model được chọn: {CONFIG['model_name']} (tối ưu cho mobile)")
print(f"📊 Max sequence length: {CONFIG['max_seq_length']}")
print(f"🎯 Target: {CONFIG['target_device']} deployment")


In [ ]:
# === IMAGE PROCESSING UTILITIES ===

def url_to_image(url: str, timeout: int = 10) -> Optional[Image.Image]:
    """Download và convert URL thành PIL Image."""
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        image = Image.open(io.BytesIO(response.content)).convert("RGB")
        return image
    except (requests.exceptions.RequestException, IOError) as e:
        print(f"Failed to load image from {url}: {e}")
        return None

def extract_image_urls_from_markdown(text: str) -> Tuple[str, List[str]]:
    """Extract image URLs từ markdown text và thay thế bằng placeholders."""
    # Pattern cho markdown images: ![alt](url)
    image_pattern = r"!\[.*?\]\((.*?)\)"
    image_urls = re.findall(image_pattern, text)
    # Remove image markdown syntax
    cleaned_text = re.sub(image_pattern, " [IMAGE] ", text).strip()
    return cleaned_text, image_urls

def process_markdown_for_model(text: str) -> Tuple[str, List[Image.Image]]:
    """Process markdown text để extract text và images cho multimodal model."""
    cleaned_text, image_urls = extract_image_urls_from_markdown(text)
    # Download images
    images = []
    for url in image_urls:
        image = url_to_image(url)
        if image:
            images.append(image)
        else:
            print(f"⚠️ Failed to load image from {url}")
    return cleaned_text, images

print("✅ Image processing utilities loaded")


In [ ]:
# === DATASET PROCESSING ===

def create_conversation_content(text: str, images: List[Image.Image]) -> List[Dict[str, Any]]:
    """Tạo conversation content list với text và images."""
    content = []
    # Add text content nếu có
    if text.strip():
        content.append({"type": "text", "text": text.strip()})
    # Add images
    for image in images:
        content.append({"type": "image", "image": image})
    # Ensure có ít nhất một content item
    if not content:
        content.append({"type": "text", "text": ""})
    return content

def process_math_sample(sample: Dict[str, str]) -> Dict[str, List[Dict[str, Any]]]:
    """Process một math problem sample thành conversation format.
    
    FIXED: Dataset có structure ['question', 'solution'] - revert lại
    """
    try:
        # Process question
        question_text, question_images = process_markdown_for_model(sample["question"])
        user_content = create_conversation_content(question_text, question_images)
        
        # Process solution (REVERTED: dataset thực sự có 'solution' chứ không phải 'answer')
        solution_text, solution_images = process_markdown_for_model(sample["solution"])
        assistant_content = create_conversation_content(solution_text, solution_images)
        
        # Create conversation
        conversations = [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ]
        
        return {"conversations": conversations}
        
    except KeyError as e:
        print(f"❌ Missing key in sample: {e}")
        print(f"Available keys: {list(sample.keys())}")
        # Fallback với empty content
        return {
            "conversations": [
                {"role": "user", "content": [{"type": "text", "text": "Error processing sample"}]},
                {"role": "assistant", "content": [{"type": "text", "text": "Sorry, I cannot process this question."}]}
            ]
        }
    except Exception as e:
        print(f"❌ Error processing sample: {e}")
        return {
            "conversations": [
                {"role": "user", "content": [{"type": "text", "text": "Error processing sample"}]},
                {"role": "assistant", "content": [{"type": "text", "text": "Sorry, I cannot process this question."}]}
            ]
        }

def prepare_dataset(dataset_name: str, split: str, max_samples: Optional[int] = None) -> Dataset:
    """Load và prepare math dataset với improved error handling."""
    print(f"📥 Loading dataset: {dataset_name}, split: {split}")
    
    try:
        # Load dataset
        raw_dataset = load_dataset(dataset_name, split=split)
        
        if max_samples:
            raw_dataset = raw_dataset.select(range(min(max_samples, len(raw_dataset))))
        
        print(f"📊 Dataset size: {len(raw_dataset)}")
        
        # Kiểm tra structure của dataset
        if len(raw_dataset) > 0:
            sample_keys = list(raw_dataset[0].keys())
            print(f"📋 Dataset columns: {sample_keys}")
            
            # Verify expected keys
            required_keys = ["question", "answer"]
            missing_keys = [key for key in required_keys if key not in sample_keys]
            if missing_keys:
                print(f"⚠️ Warning: Missing required keys: {missing_keys}")
                print(f"Available keys: {sample_keys}")
        
        print(f"🔄 Processing {len(raw_dataset)} samples...")
        processed_data = []
        
        for i, sample in enumerate(raw_dataset):
            try:
                processed_sample = process_math_sample(sample)
                processed_data.append(processed_sample)
                
                if (i + 1) % 50 == 0:
                    print(f"✅ Processed {i + 1}/{len(raw_dataset)} samples")
            
            except Exception as e:
                print(f"❌ Error processing sample {i}: {e}")
                continue
        
        print(f"✅ Successfully processed {len(processed_data)} samples")
        return Dataset.from_list(processed_data)
        
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        raise

print("✅ Dataset processing functions loaded")


In [ ]:
# === SIMPLIFIED DATA COLLATOR ===

class SimplifiedVisionDataCollator:
    """Simplified data collator cho multimodal data. Dễ hiểu và maintain hơn."""
    
    def __init__(self, processor):
        self.processor = processor
        self.placeholder_image = None
    
    def _create_placeholder_image(self):
        """Tạo placeholder image cho text-only samples."""
        if self.placeholder_image is None:
            self.placeholder_image = Image.new('RGB', (32, 32), color=(245, 245, 245))
        return self.placeholder_image
    
    def _extract_images_from_conversation(self, conv):
        """Extract tất cả images từ conversation."""
        images = []
        for message in conv:
            for content in message.get("content", []):
                if content.get("type") == "image" and "image" in content:
                    img = content["image"]
                    if img and hasattr(img, 'convert'):
                        try:
                            img = img.convert('RGB')
                            if img.size[0] > 0 and img.size[1] > 0:
                                images.append(img)
                        except Exception:
                            continue
        return images
    
    def __call__(self, examples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        """Collate batch của examples."""
        try:
            print(f"🔄 Processing batch of {len(examples)} examples...")
            
            texts = []
            images_list = []
            
            for idx, example in enumerate(examples):
                conv = example["conversations"]
                
                # Extract images
                images = self._extract_images_from_conversation(conv)
                
                # Generate text using chat template
                text = self.processor.apply_chat_template(
                    conv, tokenize=False, add_generation_prompt=False
                )
                
                # Handle image tokens
                image_token_count = text.count('<image>')
                actual_image_count = len(images)
                
                # Ensure consistency giữa image tokens và actual images
                if actual_image_count == 0:
                    # Text-only: add placeholder image và token nếu cần
                    if image_token_count == 0:
                        # Add một image token ở đầu user message
                        text = text.replace('<|user|>', '<|user|>\\n<image>')
                    images = [self._create_placeholder_image()]
                elif image_token_count < actual_image_count:
                    # Truncate images
                    images = images[:max(1, image_token_count)]
                elif image_token_count > actual_image_count:
                    # Add placeholder images
                    while len(images) < image_token_count:
                        images.append(self._create_placeholder_image())
                
                texts.append(text)
                images_list.append(images)
                
                print(f"Example {idx}: {len(images)} images, {text.count('<image>')} tokens")
            
            # Process với processor
            print("📤 Sending to processor...")
            batch = self.processor(
                text=texts,
                images=images_list,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=CONFIG["max_seq_length"]
            )
            
            # Create labels
            labels = batch["input_ids"].clone()
            labels[labels == self.processor.tokenizer.pad_token_id] = -100
            batch["labels"] = labels
            
            print(f"✅ Batch created: {batch.keys()}")
            if "pixel_values" in batch:
                print(f"📸 pixel_values shape: {batch['pixel_values'].shape}")
            print(f"📝 input_ids shape: {batch['input_ids'].shape}")
            
            return batch
            
        except Exception as e:
            print(f"❌ Error in data collator: {e}")
            import traceback
            traceback.print_exc()
            raise e

print("✅ Simplified data collator loaded")


In [ ]:
# === MODEL SETUP VÀ TRAINING FUNCTIONS ===

def setup_model_and_processor(config: Dict[str, Any]):
    """Load và setup Gemma3N model và processor."""
    print(f"🤖 Loading {config['model_name']} model and processor...")
    
    try:
        # Load model và processor
        model, processor = FastVisionModel.from_pretrained(
            config["model_name"],
            max_seq_length=config["max_seq_length"],
            load_in_4bit=config["load_in_4bit"],
            use_gradient_checkpointing="unsloth" if config["use_gradient_checkpointing"] else False,
        )
        
        # Apply LoRA với mobile-optimized settings
        model = FastVisionModel.get_peft_model(
            model,
            finetune_vision_layers=True,
            finetune_language_layers=True,
            finetune_attention_modules=True,
            finetune_mlp_modules=True,
            r=config["lora_r"],
            lora_alpha=config["lora_alpha"],
            lora_dropout=config["lora_dropout"],
            bias="none",
            random_state=config["seed"],
            use_rslora=False,
            target_modules="all-linear",
            modules_to_save=["lm_head", "embed_tokens"],
        )
        
        # Setup chat template
        processor = get_chat_template(processor, "gemma-3n")
        
        print("✅ Model and processor setup complete!")
        return model, processor
        
    except Exception as e:
        print(f"❌ Error setting up model: {e}")
        raise

def setup_comet_ml(config: Dict[str, Any]) -> Optional[object]:
    """Setup Comet ML experiment tracking."""
    if not COMET_AVAILABLE or config["report_to"] != "comet_ml":
        print("📊 Using tensorboard for logging")
        return None
    
    try:
        experiment = comet_ml.Experiment(
            workspace=COMET_CONFIG.get("workspace"),
            project_name=COMET_CONFIG.get("project"),
            auto_metric_logging=True,
            auto_param_logging=True,
        )
        
        # Log configuration
        experiment.log_parameters(config)
        
        # Add tags
        for tag in COMET_CONFIG.get("tags", []):
            experiment.add_tag(tag)
        
        print(f"✅ Comet ML experiment initialized: {experiment.url}")
        return experiment
        
    except Exception as e:
        print(f"❌ Failed to initialize Comet ML: {e}")
        print("📊 Falling back to tensorboard")
        config["report_to"] = "tensorboard"
        return None

def create_trainer(model, processor, train_dataset, config: Dict[str, Any]):
    """Create mobile-optimized SFTTrainer."""
    # Enable training
    FastVisionModel.for_training(model)
    
    # Create simplified data collator
    data_collator = SimplifiedVisionDataCollator(processor)
    
    # Mobile-optimized training arguments
    training_args = SFTConfig(
        # Basic training settings
        output_dir=config["output_dir"],
        max_steps=config["max_steps"],
        per_device_train_batch_size=config["per_device_train_batch_size"],
        gradient_accumulation_steps=config["gradient_accumulation_steps"],
        
        # Optimization settings
        learning_rate=config["learning_rate"],
        warmup_ratio=config["warmup_ratio"],
        weight_decay=config["weight_decay"],
        optim="adamw_torch_fused",
        lr_scheduler_type="cosine",
        
        # Memory optimization cho mobile
        gradient_checkpointing=config["use_gradient_checkpointing"],
        gradient_checkpointing_kwargs={"use_reentrant": False} if config["use_gradient_checkpointing"] else {},
        max_grad_norm=0.3,
        dataloader_pin_memory=False,  # Tối ưu cho mobile
        
        # Logging và saving
        logging_steps=config["logging_steps"],
        save_strategy="steps",
        save_steps=config["save_steps"],
        report_to=config["report_to"],
        
        # Vision-specific settings
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        max_length=config["max_seq_length"],
        
        # Reproducibility
        seed=config["seed"],
        
        # Mobile optimization
        fp16=True,  # Use fp16 cho mobile efficiency
        dataloader_num_workers=1,  # Giảm workers
    )
    
    # Create trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        processing_class=processor.tokenizer,
        data_collator=data_collator,
        args=training_args,
    )
    
    return trainer

print("✅ Training functions loaded")


In [ ]:
# === TRAINING PIPELINE ===

print("🚀 STARTING GEMMA 3N MOBILE-OPTIMIZED TRAINING PIPELINE")
print("="*70)

# 1. Create output directory
os.makedirs(CONFIG["output_dir"], exist_ok=True)
print(f"📁 Output directory: {CONFIG['output_dir']}")

# 2. Setup Comet ML (optional)
comet_experiment = setup_comet_ml(CONFIG)

# 3. Setup model và processor
print("\n🚀 Step 1: Setting up model and processor...")
model, processor = setup_model_and_processor(CONFIG)

# 4. Prepare dataset
print("\n🚀 Step 2: Preparing dataset...")
# Load một subset nhỏ để test trước (có thể thay đổi max_samples=None để load toàn bộ)
train_dataset = prepare_dataset(
    CONFIG["dataset_name"], 
    CONFIG["train_split"],
    max_samples=50  # Test với 50 samples trước, sau đó có thể bỏ parameter này
)

# Dataset statistics
print(f"\n📊 Dataset Statistics:")
print(f"   - Training samples: {len(train_dataset)}")

# Count samples với images
samples_with_images = 0
for sample in train_dataset:
    for conv in sample["conversations"]:
        for content in conv.get("content", []):
            if content.get("type") == "image":
                samples_with_images += 1
                break
        else:
            continue
        break

print(f"   - Samples with images: {samples_with_images}")
print(f"   - Text-only samples: {len(train_dataset) - samples_with_images}")

# 5. Create trainer
print("\n🚀 Step 3: Creating trainer...")
trainer = create_trainer(model, processor, train_dataset, CONFIG)

# 6. Test data collator trước khi training
print("\n🧪 Testing data collator với sample batch...")
try:
    test_samples = [train_dataset[i] for i in range(min(2, len(train_dataset)))]
    test_batch = trainer.data_collator(test_samples)
    print("✅ Data collator test passed!")
    print(f"   Batch keys: {list(test_batch.keys())}")
    for key, value in test_batch.items():
        if hasattr(value, 'shape'):
            print(f"   {key}: {value.shape}")
except Exception as e:
    print(f"❌ Data collator test failed: {e}")
    print("Please check the dataset format and try again.")
    raise

# 7. Start training
print("\n🚀 Step 4: Starting training...")
print(f"   📱 Model: {CONFIG['model_name']} (optimized for mobile)")
print(f"   📁 Output directory: {CONFIG['output_dir']}")
print(f"   🎯 Max steps: {CONFIG['max_steps']}")
print(f"   📦 Batch size: {CONFIG['per_device_train_batch_size']}")
print(f"   🔄 Gradient accumulation: {CONFIG['gradient_accumulation_steps']}")
print(f"   💪 Effective batch size: {CONFIG['per_device_train_batch_size'] * CONFIG['gradient_accumulation_steps']}")

# Train the model
try:
    trainer_stats = trainer.train()
    print("\n✅ Training completed successfully!")
    print(f"   Final loss: {trainer_stats.training_loss:.4f}")
except Exception as e:
    print(f"\n❌ Training failed: {e}")
    import traceback
    traceback.print_exc()
    raise

# 8. Save model cho mobile deployment
print("\n💾 Saving model for mobile deployment...")

try:
    # Save LoRA adapters
    lora_save_path = f"{CONFIG['output_dir']}/lora_adapters"
    model.save_pretrained(lora_save_path)
    processor.tokenizer.save_pretrained(lora_save_path)
    print(f"✅ LoRA adapters saved to: {lora_save_path}")
    
    # Save merged model for inference
    merged_save_path = f"{CONFIG['output_dir']}/merged_model"
    model.save_pretrained_merged(merged_save_path, processor.tokenizer, save_method="merged_16bit")
    print(f"✅ Merged model saved to: {merged_save_path}")
    
    # Save mobile-optimized GGUF format (optional)
    try:
        gguf_save_path = f"{CONFIG['output_dir']}/gguf_model"
        model.save_pretrained_gguf(gguf_save_path, processor.tokenizer, quantization_method="q4_k_m")
        print(f"✅ GGUF model saved to: {gguf_save_path}")
    except Exception as e:
        print(f"⚠️ GGUF export failed (optional): {e}")
    
    print("\n🎉 All models saved successfully!")
    print(f"📱 Ready for mobile deployment!")
    
except Exception as e:
    print(f"❌ Error saving models: {e}")
    import traceback
    traceback.print_exc()

# 9. Test model sau khi training
print("\n🧪 Testing fine-tuned model...")

try:
    # Enable inference mode
    FastVisionModel.for_inference(model)
    
    # Test prompt
    test_prompt = """
<|user|>
Bài toán: Một hình chữ nhật có chiều dài 12cm và chiều rộng 8cm. Tính diện tích của hình chữ nhật này.
<|assistant|>
"""
    
    # Tokenize input
    inputs = processor.tokenizer(
        test_prompt.strip(),
        return_tensors="pt",
        truncation=True,
        max_length=CONFIG["max_seq_length"]
    )
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    
    # Decode response
    response = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("📝 Test Input:")
    print(test_prompt.strip())
    print("\n🤖 Model Response:")
    print(response[len(test_prompt.strip()):])
    
    print("\n✅ Model test completed!")
    
except Exception as e:
    print(f"❌ Model test failed: {e}")
    import traceback
    traceback.print_exc()

# Final summary
print("\n" + "="*60)
print("🎉 TRAINING SUMMARY")
print("="*60)
print(f"📱 Model: {CONFIG['model_name']} (Mobile Optimized)")
print(f"📊 Dataset: {CONFIG['dataset_name']}")
print(f"🎯 Training steps: {CONFIG['max_steps']}")
print(f"💾 Output directory: {CONFIG['output_dir']}")
print(f"🔧 LoRA rank: {CONFIG['lora_r']}")
print(f"📏 Max sequence length: {CONFIG['max_seq_length']}")

print("\n📁 Saved Models:")
print(f"   - LoRA adapters: {CONFIG['output_dir']}/lora_adapters")
print(f"   - Merged model: {CONFIG['output_dir']}/merged_model")
print(f"   - GGUF model: {CONFIG['output_dir']}/gguf_model (if available)")

print("\n🚀 Next Steps:")
print("   1. Test model với real data")
print("   2. Convert to mobile format (TensorFlow Lite, Core ML, etc.)")
print("   3. Deploy to mobile app")
print("   4. Optimize inference performance")
print("   5. Add evaluation metrics")

print("\n💡 Mobile Deployment Tips:")
print("   - Sử dụng GGUF format cho llama.cpp")
print("   - Consider quantization để giảm model size")
print("   - Test trên target mobile devices")
print("   - Monitor memory usage và inference speed")

print("\n✅ Training completed successfully!")
print("="*60)

# Cleanup Comet experiment
if comet_experiment:
    try:
        comet_experiment.end()
        print("📊 Comet ML experiment ended")
    except:
        pass
